In [ ]:
%pylab inline

# ``TfData`` objects

A ``TfData`` is a ``MaskedArray`` dedicated to handle Time-Frequency representations obtained from a real signal. As such, it has  two attributes ``tf_params`` and ``signal_params``, giving respectively the parameters of the STFT used to obtain the representation, and the parameters of the real signali, as well as dedicated methods to facilitate the manipulation of time-frequency data.

In [ ]:
from pyteuf import TfData, Stft
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (14, 4)

# Data parameters
tf_params = {'hop': 32, 'n_bins': 128, 'win_len': 64, 'win_name': 'hanning'}
signal_params = {'len':1024, 'fs':44100}

## Example of TF representation of a complex signal, with binary masking

As for ``MaskedArray``, ``TfData`` can be initialized from a 2D complex nd-array with or without mask. Parameters ``stft_params`` and ``signal_params`` are explicitly given if available. For a complex signal, coefficients for both negative and positive frequencies are handled and displayed

In [ ]:
# Data ans mask for a complex signal (no symetry in TF)
tf_shape_complex = (tf_params['n_bins'], signal_params['len'] // tf_params['hop'])
mask_complex = np.zeros(tf_shape_complex)
mask_complex[int(0.2*tf_shape_complex[0]):int(0.6*tf_shape_complex[0]), 
             int(0.25*tf_shape_complex[1]):int(0.7*tf_shape_complex[1])] = True
data_complex = np.random.randn(*tf_shape_complex) + 1j * np.random.randn(*tf_shape_complex)

# TF data
X = TfData(data=data_complex, mask=mask_complex, tf_params=tf_params, signal_params=signal_params)

plt.subplot(121)
X.plot_spectrogram()
plt.subplot(122)
X.plot_mask()
print(X)

## Example of TF representation of a real signal, with binary masking

For a real signal, only the non-negative frequencies area is handled due to Hermitian symetry.

In [ ]:
# Data and mask for a real signal (Hermitian symetry in TF)
tf_shape_real = (tf_params['n_bins'] // 2 + 1, signal_params['len'] // tf_params['hop'])
mask_real = np.zeros(tf_shape_real)
mask_real[int(0.2*tf_shape_real[0]):int(0.6*tf_shape_real[0]), 
          int(0.25*tf_shape_real[1]):int(0.7*tf_shape_real[1])] = True
data_real = np.random.randn(*tf_shape_real) + 1j * np.random.randn(*tf_shape_real)

# TF data
X = TfData(data=data_real, mask=mask_real, tf_params=tf_params, signal_params=signal_params)

plt.subplot(121)
X.plot_spectrogram()
plt.subplot(122)
X.plot_mask()
print(X)

## Example of TF representation with complex masking

With complex masking, two masks are handled: a mask for amplitudes and a mask for phases.

In [ ]:
# Data and mask for a real signal (Hermitian symetry in TF)
tf_shape_real = (tf_params['n_bins'] // 2 + 1, signal_params['len'] // tf_params['hop'])
mask_mag = np.zeros(tf_shape_real)
mask_mag[int(0.2*tf_shape_real[0]):int(0.6*tf_shape_real[0]), 
         int(0.25*tf_shape_real[1]):int(0.7*tf_shape_real[1])] = True
mask_phi = np.zeros(tf_shape_real)
mask_phi[int(0.5*tf_shape_real[0]):int(0.65*tf_shape_real[0]), 
         int(0.65*tf_shape_real[1]):int(0.75*tf_shape_real[1])] = True
data_real = np.random.randn(*tf_shape_real) + 1j * np.random.randn(*tf_shape_real)

# TF data
X = TfData(data=data_real,
           mask_magnitude=mask_mag, mask_phase=mask_phi,
           tf_params=tf_params, signal_params=signal_params)
print(X)

One may select one of the masks or a combination of them for display.

In [ ]:
for mask_type in ['any', 'all', 'magnitude', 'phase', 'magnitude only', 'phase only']:
    plt.figure()
    plt.subplot(121)
    X.plot_spectrogram(mask_type=mask_type)
    plt.title('mask type: {}'.format(mask_type))
    plt.subplot(122)
    X.plot_mask(mask_type=mask_type)
    plt.title('mask type: {}'.format(mask_type))


Selecting a mask type can also be used to extract the related mask:

In [ ]:
for mask_type in ['any', 'all', 'magnitude', 'phase', 'magnitude only', 'phase only']:
    mask = X.get_unknown_mask(mask_type)
    print('Ratio of "{}" unknown coefficients: {:.1%}'.format(mask_type, np.mean(mask)))
    mask = X.get_known_mask(mask_type)
    print('Ratio of "{}" known coefficients: {:.1%}'.format(mask_type, np.mean(mask)))

Other useful properties:

In [ ]:
print('Number of frequencies: {}'.format(X.n_frequencies))
print('Number of frames: {}'.format(X.n_frames))
print('Start time of frames: {}'.format(X.start_times))
print('Center time of frames: {}'.format(X.mid_times))
print('End time of frames: {}'.format(X.end_times))

Current operations between two ``TfData`` objects are not recommended since resulting operations on masks are not designed adequately:

In [ ]:
mask1 = np.zeros(tf_shape_real)
mask1[int(0.2*tf_shape_real[0]):int(0.6*tf_shape_real[0]), 
      int(0.25*tf_shape_real[1]):int(0.7*tf_shape_real[1])] = True
mask2 = np.zeros(tf_shape_real)
mask2[int(0.5*tf_shape_real[0]):int(0.65*tf_shape_real[0]), 
      int(0.65*tf_shape_real[1]):int(0.75*tf_shape_real[1])] = True
data_real = np.random.randn(*tf_shape_real) + 1j * np.random.randn(*tf_shape_real)

# TF data
X1 = TfData(data=np.random.randn(*tf_shape_real) + 1j * np.random.randn(*tf_shape_real),
            mask=mask1, tf_params=tf_params, signal_params=signal_params)
X2 = TfData(data=np.random.randn(*tf_shape_real) + 1j * np.random.randn(*tf_shape_real),
            mask=mask2, tf_params=tf_params, signal_params=signal_params)

plt.figure()
plt.subplot(121)
X1.plot_spectrogram()
plt.subplot(122)
X2.plot_spectrogram()

plt.figure()
X3 = X1 + X2
X4 = X1 - X2
plt.subplot(121)
X3.plot_spectrogram()
plt.subplot(122)
X4.plot_spectrogram()

plt.figure()
X3 = X1 * X2
X4 = X1 / X2
plt.subplot(121)
X3.plot_spectrogram()
plt.subplot(122)
X4.plot_spectrogram()
pass